In [5]:
import pandas as pd
import plotly.express as px

# Load your Excel file
df = pd.read_excel('/Bird_Migration_Data.xlsx')

# Filter relevant columns
df_filtered = df[['decimalLatitude', 'decimalLongitude', 'species', 'month']]

# Remove rows with missing latitude or longitude values
df_filtered = df_filtered.dropna(subset=['decimalLatitude', 'decimalLongitude'])
df_filtered = df_filtered.sort_values(by='month')
# Group by species and month to calculate mean latitudes and longitudes
df_grouped = df_filtered.groupby(['species', 'month'], as_index=False).agg({
    'decimalLatitude': 'mean',
    'decimalLongitude': 'mean'
})
df_grouped['month'] = df_grouped['month'].astype(int)
df_grouped = df_grouped.sort_values(by='month')

In [7]:
# Determine direction of movement based on latitude difference
df_grouped['direction'] = df_grouped.groupby('species')['decimalLatitude'].diff()

# Assign colors based on direction (upward = green, downward = blue)
df_grouped['color'] = df_grouped['direction'].apply(lambda x: 'green' if x > 0 else 'blue')

# Create the plot
fig = px.scatter_geo(
    df_grouped,
    lat='decimalLatitude',
    lon='decimalLongitude',
    color='color',  # Colors based on direction
    color_discrete_map={'green': 'green', 'blue': 'blue'},
    hover_name='species',
    hover_data=['month', 'direction'],
    animation_frame='month',
    projection='natural earth',
    title="Migration Patterns",
    template='plotly',
    size_max=10,
    opacity=0.8
)

fig.update_geos(
    showcoastlines=True,
    coastlinecolor="Black",
    showland=True,
    landcolor="lightgray",
    showcountries=True,
    countrycolor="darkgray"
)

fig.show()
